# 10차시 모평균 비교에 관한 가설검정: One way ANOVA

## 01 일원 분산 분석(One Way ANOVA) 개요

### 일원 분산 분석의 특징

- 수치형 종속변수와 명목형 독립변수가 각각 1개 일 때 실시하는 분석
- 종속변수로 나뉘어지는 그룹이 2개 이상일 때 사용
- 귀무가설을 기각하는 경우 사후 검정(Post-hoc)을 실시

### 가설

- 귀무가설(H0): 집단 간 평균이 같음
- 대립가설(H1): 평균이 같지 않은 집단이 한 쌍 이상 존재

### 일원 분산 분석의 사후 검정

- 사후 검정 방법으로는 Tukey's HSD, Duncan's MRT, Scheffe's test가 있다.
- 실습 해볼 사후 검정은 Tukey's HSD이다.
- 모든 집단 간 독립 2 표본 t-검정을 하는 것과 유사하며 어던 집단 간 평균이 유의하게 차이 나는지 알 수 있다.

## 주요 함수 및 메서드 소개

### 02 일원 분산 분석 - f_oneway()

- 일원 분산 분석을 수행하는 scipy의 함수
- 입력은 각각의 집단을 pandas의 series로 하는 것을 권장
- 출력은 검정통계량과 p-value 두 개 이며 하나의 객체에 할당할 경우 튜플로 저장됨

In [2]:
import pandas as pd
from scipy.stats import f_oneway

In [3]:
df = pd.read_csv("강의자료/실습파일/diamonds.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df["color"].unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [5]:
stat, p = f_oneway(df.loc[df["color"] == "E", "price"],
                  df.loc[df["color"] == "I", "price"],
                  df.loc[df["color"] == "J", "price"])
print(stat.round(3))
print(p.round(3))

621.846
0.0


In [6]:
df.groupby("color")["price"].mean().reset_index()

,color,price
0,D,3169.954096
1,E,3076.752475
2,F,3724.886397
3,G,3999.135671
4,H,4486.669196
5,I,5091.874954
6,J,5323.818020


### 일원 분산 분석 - ols(), anova_lm()

- statsmodels의 일원 분산 분석을 수행하는 함수
- ols() 함수는 모델을 생성하고 적합
- anova_lm() 함수는 적합된 모델 정보를 기반으로 일원 분산 분석표를 보여줌
- ols() 함수에 수식 입력 시 독립변수에 C() 함수 사용 권장

In [7]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [8]:
bike = pd.read_csv("강의자료/실습파일/bike.csv")
bike.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [9]:
model = ols(formula = 'temp ~ season', data = bike).fit()  # fit: 학습
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
season,1.0,44221.657301,44221.657301,780.591754,5.879560e-166
Residual,10884.0,616594.417651,56.651453,NaN,NaN


In [10]:
model = ols(formula = 'temp ~ C(season)', data = bike).fit() # C(): Categorical, 시즌 변수는 명목형이다, 숫자형의 경우 꼭 C() 사용할 것
anova_lm(model) # F: 검정통계량 / PR: 검정통계량보다 큰 값이 나올 확률

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


### 사후검정 - pairwise_tukeyhsd()

- statsmodels의 Tukey 사후검정을 실시하는 함수
- 함수 내에 종속변수와 독립변수를 차례대로 선언
- 결과를 출력하기 위해서 print() 함수를 반드시 사용
- reject 변수의 True는 귀무가설(두 집단의 평균 차이가 유의미하게 나지 않음)을 기각하는 것

In [11]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [12]:
result = pairwise_tukeyhsd(bike["temp"], bike["season"])
print(result)  # reject: FWER 0.05 기준이 적용됨

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper   reject
-----------------------------------------------------
     1      2   10.293   0.0   9.9598  10.6262   True
     1      3  16.2586   0.0  15.9254  16.5918   True
     1      4   4.1187   0.0   3.7856   4.4519   True
     2      3   5.9656   0.0   5.6339   6.2974   True
     2      4  -6.1742   0.0   -6.506  -5.8425   True
     3      4 -12.1399   0.0 -12.4716 -11.8081   True
-----------------------------------------------------


## Q1. 분산분석 시 사용하는 함수 또는 라이브러리와 관련 없는 것은?
1. ANOVA (답)
2. f_oneway
3. ols
4. statsmodels

## Q2. 계절별 기온 평균을 분산분석 시 검정통계량은?


In [14]:
Q2 = pd.read_csv("강의자료/실습파일/bike.csv")
Q2.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [15]:
Q2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [18]:
formula = "temp ~ C(season)"  # C() 범주형 변수임을 명시
lm = ols(formula, Q2).fit()
anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


## Q3. 요일별 registered의 평균을 사후검정 했을 때 평균이 유의미하게 차이 나지 않는 조합은 총 몇개 인가?

In [19]:
Q3 = pd.read_csv("강의자료/실습파일/bike.csv")
Q3.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [21]:
Q3["datetime"] = pd.to_datetime(Q3["datetime"])
Q3["weekday"] = Q3["datetime"].dt.weekday
Q3.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,5


In [23]:
result = pairwise_tukeyhsd(Q3["registered"], Q3["weekday"])
print(result)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     0      1   6.1979  0.913   -9.719  22.1148  False
     0      2    5.343 0.9559 -10.5429  21.2289  False
     0      3  12.7424 0.2132  -3.1384  28.6232  False
     0      4   6.2956 0.9074  -9.6473  22.2386  False
     0      5 -27.5063    0.0 -43.3093 -11.7034   True
     0      6 -36.7583    0.0 -52.5736 -20.9429   True
     1      2  -0.8549    1.0 -16.7718   15.062  False
     1      3   6.5445 0.8895  -9.3673  22.4562  False
     1      4   0.0977    1.0 -15.8761  16.0715  False
     1      5 -33.7042    0.0 -49.5383 -17.8702   True
     1      6 -42.9562    0.0 -58.8026 -27.1097   True
     2      3   7.3994 0.8159  -8.4814  23.2802  False
     2      4   0.9526    1.0 -14.9903  16.8956  False
     2      5 -32.8493    0.0 -48.6523 -17.0464   True
     2      6 -42.1013    0.0 -57.9166 -26.2859   True
     3    